# import imprortant library


In [52]:
import numpy as np
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from keras.utils import to_categorical
import torch
import warnings
warnings.filterwarnings('ignore')
import torchvision.models as models
from keras.utils import to_categorical
from sklearn.model_selection import KFold 
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch import optim , autocast
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.cuda.amp import GradScaler
from torchvision import transforms, datasets
from collections import Counter
from torch.optim import AdamW
from sklearn.metrics import confusion_matrix
from torchvision.models import googlenet
from efficientnet_pytorch import EfficientNet
import torch.utils.data as data
%pylab inline
import psutil
import lightning as L
import torch.nn.functional as F
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [53]:
L.seed_everything(42)

Seed set to 42


42

# load dataset


In [54]:
wafer_tensors = torch.load('wafer_tensors.pt')
label_wm = torch.load('label_wm.pt')

In [55]:
wafer_tensors.to("cpu")
label_wm.to("cpu")

tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])

In [56]:
print("Wafer Tensors Shape:", wafer_tensors.shape)
print("Label Tensors Shape:", label_wm.shape)

Wafer Tensors Shape: torch.Size([368568, 3, 56, 56])
Label Tensors Shape: torch.Size([368568, 1, 36])


In [57]:
label = []
label_wm[0].squeeze()
classes = ['C+EL', 'C+EL+L', 'C+EL+S', 'C+ER', 'C+ER+L', 'C+ER+S', 'C+L',
       'C+L+EL+S', 'C+L+ER+S', 'C+L+S', 'C+S', 'Center', 'D+EL', 'D+EL+L',
       'D+EL+S', 'D+ER', 'D+ER+L', 'D+ER+S', 'D+L', 'D+L+EL+S',
       'D+L+ER+S', 'D+L+S', 'D+S', 'Donut', 'EL+L', 'EL+L+S', 'EL+S',
       'ER+L', 'ER+S', 'Edge-Loc', 'Edge-Ring', 'L+S', 'Loc', 'Near-full',
       'Random', 'Scratch']
label_dict = {class_name: i for i, class_name in enumerate(classes)}
label = np.argmax(label_wm.squeeze(), axis=1)

In [58]:
tensor_array = label.numpy()
unique_values, value_counts = np.unique(tensor_array, return_counts=True)

# Sort values by their counts in descending order
sorted_indices = np.argsort(value_counts)[::-1]
sorted_values = unique_values[sorted_indices]
sorted_counts = value_counts[sorted_indices]

# Print or manipulate the sorted values and their counts as needed
print(unique_values)
print(value_counts)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]
[10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 12882
 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 12000  9990
 10000 10000 10000 10000 10000 10378  9680 12045 10779  9685 10392 10737]


In [59]:
args = {"BATCH_SIZE": 512, "LEARNING_RATE": 0.0008, "NUM_EPOCH": 100}

In [60]:
DEVICE = "cpu"

In [61]:
class BasicDataset(data.Dataset):
    def __init__(self, x_tensor, y_tensor):
        super(BasicDataset, self).__init__()

        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [62]:
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(
    wafer_tensors, label_wm, test_size=0.2, random_state= 42, shuffle=True
)
train_X, val_X, train_Y, val_Y = model_selection.train_test_split(
    train_X, train_Y, test_size=0.3, random_state=42, shuffle=True
)
dataset_train = BasicDataset(train_X, train_Y)
dataset_val = BasicDataset(val_X, val_Y)
dataset_test = BasicDataset(test_X, test_Y)
dataset = ConcatDataset([dataset_train, dataset_val])

In [63]:
# Create DataLoader for training
train_loader = DataLoader(dataset_train, batch_size=args["BATCH_SIZE"], shuffle=True)

# Create DataLoader for validation
val_loader = DataLoader(dataset_val, batch_size=args["BATCH_SIZE"], shuffle=False)

# Create DataLoader for testing
test_loader = DataLoader(dataset_test, batch_size=args["BATCH_SIZE"], shuffle=False)

In [64]:
for batch in train_loader:
    x, y = batch
    print("Shape of x:", x.shape)
    print("Type of x:", type(x))
    print("Shape of y:", y.shape)
    print("Type of y:", type(y))
    break

Shape of x: torch.Size([512, 3, 56, 56])
Type of x: <class 'torch.Tensor'>
Shape of y: torch.Size([512, 1, 36])
Type of y: <class 'torch.Tensor'>


In [65]:
class GoogLeNet(L.LightningModule):
    def __init__(self, num_classes=36):
        super(GoogLeNet, self).__init__()
        self.googlenet = models.googlenet(pretrained=True)
        num_features = self.googlenet.fc.in_features
        self.googlenet.fc = nn.Linear(num_features, num_classes)
    def forward(self, x):
        x = self.googlenet(x)
        return x
    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.view(-1, 36)
        y = y.float()
        preds = self.googlenet(x)
        preds = preds.view(-1, 36)
        loss = F.cross_entropy(preds, y)
        acc = (preds.argmax(dim=-1) == y).float().mean()
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.view(-1, 36)
        y = y.float()
        preds = self.googlenet(x)
        preds = preds.view(-1, 36)
        val_loss = F.cross_entropy(preds, y)
        acc = (preds.argmax(dim=-1) == y).float().mean()
        self.log("val_acc", acc)
        self.log("val_loss", val_loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.view(-1, 36)
        y = y.float()
        y_hat = self.googlenet(x)
        y_hat = y_hat.view(-1, 36)
        test_loss = F.cross_entropy(y_hat, y)
        self.log("test_loss", test_loss)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.googlenet.parameters(), lr=1e-3)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]    

In [66]:
early_stop_callback = EarlyStopping(monitor="val_accuracy", min_delta=0.00, patience=3, verbose=False, mode="max")

In [67]:
model = GoogLeNet(num_classes=36)
trainer = L.Trainer(max_epochs=10,callbacks=[early_stop_callback],accelerator="auto", devices="auto")
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | googlenet | GoogLeNet | 5.6 M 
----------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.547    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (512) must match the size of tensor b (36) at non-singleton dimension 1

In [50]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/144 [00:13<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported